# What is REST API and why to use api?

[REST](https://www.ics.uci.edu/~fielding/pubs/dissertation/rest_arch_style.htm): **Representational State Transfer**

- Architectural style for communication often used in Web service development.
- Uses a stateless operation in client server protocol to manipulate  server resources(image, blog etc.) in the web.

When using [http](https://www.ietf.org/rfc/rfc2616.txt)/https most common available operations are



**GET**: To get the data from server

**POST**: To submit data

**PUT**: to update a resource

**DELETE**: delete the resource


Response will have a payload formatted in HTML, XML, **JSON**, or some other format.

# JSON (Java Script object notation)
json is primary format used for data exchange. python has json library. we can use it to convert python object to string(**dumps method**) and  vice -versa **loads method**.

# Status code for GET request
- 200: success
- 400: bad request
- 401: authentication failure
- 404: resouce not found. Like wrong endpoint

Client applications use URI/URL to access RESTFUL web services.

REST API has __*base url + endpoint+ parameters*__

e.g - https://api.github.com/users
    - https://api.github.com/users/user_id/repos

**Before using python Requests module, let's use curl to see rest API in action**

Using github developer API we can get list of users as

In [ ]:
!curl https://api.github.com/users

# [Requests: HTTP for Humans](http://docs.python-requests.org/en/master/)

From the website

**"*Requests allows you to send organic, grass-fed HTTP/1.1 requests, without the need for manual labor.*"**

Let's try to get NASA data using web API

In [3]:
import requests

## APOD( Astronomy Picture of the Day)
For passing parameters to the API using python dictionary

In [ ]:
parameters = {"api_key":"DEMO_KEY"}
response = requests.get("https://api.nasa.gov/planetary/apod", params=parameters)

In [ ]:
print(response.status_code)

Codes are available as

In [ ]:
requests.codes.ok

In [ ]:
response.headers

In [ ]:
response.text

In [ ]:
json_data = response.json()
type(json_data)

In [ ]:
#!pip install pillow

In [ ]:
from PIL import Image
from io import BytesIO
response = requests.get(json_data['url'])
img = Image.open(BytesIO(response.content))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


In [ ]:
fig, ax = plt.subplots(figsize=(14, 14))
ax.imshow(img)

## Mars rovers photo

In [ ]:
parameters = {"api_key":"DEMO_KEY", "sol":1000, "camera":"fhaz"}
api_end_point= "https://api.nasa.gov/mars-photos/api/v1/rovers/curiosity/photos"
response = requests.get(api_end_point, params=parameters)
response.status_code

In [ ]:
json_data = response.json()
json_data

In [ ]:

for image_data in json_data['photos']:
    response = requests.get(image_data['img_src'])
    fig, ax = plt.subplots(figsize=(14, 14))
    img = Image.open(BytesIO(response.content))
    ax.imshow(img)

# Twitter example with POST and GET

1. create new app at https://apps.twitter.com/app/new
1. [twitter api reference index](https://developer.twitter.com/en/docs/api-reference-index)


# POST oauth2/token
Allows a registered application to obtain an OAuth 2 Bearer Token, which can be used to make API requests on an application's own behalf, without a user context.
https://developer.twitter.com/en/docs/basics/authentication/api-reference/token


In [5]:
client_key = 'cMAKpOIAiLXtsYsvYHSyy4ISJ'
client_secret = 'sqH2RBkSrX4I0GmqGb7uLAausxHVaUg86umAIyQ4fjPtwF4bmR'

In [6]:
import base64

key_secret = '{}:{}'.format(client_key, client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

In [7]:
base_url = 'https://api.twitter.com/'
auth_endpoint = base_url+'oauth2/token'

auth_headers = { 'Authorization': 'Basic {}'.format(b64_encoded_key),
                'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'}

auth_data = { 'grant_type': 'client_credentials'}



Let's post to authentication endpoint and get the token to be used in subsequent REST API call

In [9]:
response = requests.post(auth_endpoint, headers=auth_headers, data=auth_data)
response.status_code

200

In [10]:
json_data =  response.json()
json_data

{'access_token': 'AAAAAAAAAAAAAAAAAAAAAEGe9QAAAAAALnCjfdkTP%2FJGQxEZOPTqGQL7jUs%3DTVrscEzlzvKPhNerXde7Btkx2aJ2TYtpuCpaHC3TwRfIKnX4vL',
 'token_type': 'bearer'}

In [11]:
access_token = json_data['access_token']

# Let's do some search on climate change
This is how we got search endpoint

https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets

In [12]:
search_headers = {'Authorization': 'Bearer {}'.format(access_token)    
}

parameters = { 'q': 'climate change',
                    'result_type': 'recent',
                'count': 5 }

search_url = base_url+'1.1/search/tweets.json'

response = requests.get(search_url, headers=search_headers, params=parameters)

In [ ]:
json_tweet_data = response.json()


In [17]:
import pprint

In [18]:
pprint.pprint(json_tweet_data)

{'search_metadata': {'completed_in': 0.029,
                     'count': 5,
                     'max_id': 1094034863525515264,
                     'max_id_str': '1094034863525515264',
                     'next_results': '?max_id=1094034851148099583&q=climate%20change&count=5&include_entities=1&result_type=recent',
                     'query': 'climate+change',
                     'refresh_url': '?since_id=1094034863525515264&q=climate%20change&result_type=recent&include_entities=1',
                     'since_id': 0,
                     'since_id_str': '0'},
 'statuses': [{'contributors': None,
               'coordinates': None,
               'created_at': 'Sat Feb 09 00:46:52 +0000 2019',
               'entities': {'hashtags': [],
                            'symbols': [],
                            'urls': [],
                            'user_mentions': [{'id': 81741855,
                                               'id_str': '81741855',
                                

In [15]:
for status in json_tweet_data['statuses']:
    print(status['text'] + '\n')

RT @StevenBeschloss: At a time of great challenge, from climate change to gun violence to income equality, what a waste of time dealing w/T…

RT @SenSanders: I am proud to be an original co-sponsor of the Green New Deal proposal. We must address the existential crisis of planetary…

RT @brianschatz: I’m not a socialist. But I believe that every American deserves health care. I also believe that college should be debt fr…

RT @ElaineEDO: 'Delicious irony': How miner's challenge became a climate change win 

This piece by @p_hannam explains how a case originall…

Climate change caused by man is a hoax. What a brainwashed idiot you appear to be, man from the hand. https://t.co/0nIs11zubo

